# ജനറേറ്റീവ് നെറ്റ്വർക്കുകൾ

റികറന്റ് ന്യൂറൽ നെറ്റ്വർക്കുകൾ (RNNs) மற்றும் അവയുടെ ഗേറ്റഡ് സെൽ വകഭേദങ്ങൾ, ഉദാഹരണത്തിന് ലോങ് ഷോർട്ട് ടേം മെമ്മറി സെലുകൾ (LSTMs) மற்றும் ഗേറ്റഡ് റികറന്റ് യൂണിറ്റുകൾ (GRUs), ഭാഷാ മോഡലിംഗിന് ഒരു സംവിധാനം നൽകുന്നു, അതായത് അവ വാക്കുകളുടെ ക്രമീകരണം പഠിച്ച് ഒരു സീക്വൻസിലെ അടുത്ത വാക്ക് പ്രവചിക്കാൻ കഴിയും. ഇതിലൂടെ RNN-കൾ **ജനറേറ്റീവ് ടാസ്കുകൾ**ക്കായി ഉപയോഗിക്കാം, ഉദാഹരണത്തിന് സാധാരണ ടെക്സ്റ്റ് ജനറേഷൻ, മെഷീൻ ട്രാൻസ്ലേഷൻ,甚至 ഇമേജ് ക്യാപ്ഷനിംഗ്.

മുൻ യൂണിറ്റിൽ ചർച്ച ചെയ്ത RNN ആർക്കിടെക്ചറിൽ, ഓരോ RNN യൂണിറ്റും അടുത്ത ഹിഡൻ സ്റ്റേറ്റ് ഔട്ട്പുട്ടായി ഉൽപ്പാദിപ്പിച്ചിരുന്നു. എന്നാൽ, ഓരോ റികറന്റ് യൂണിറ്റിനും മറ്റൊരു ഔട്ട്പുട്ട് കൂടി ചേർക്കാം, ഇത് ഒരു **സീക്വൻസ്** (മൂല സീക്വൻസിന്റെ നീളത്തിന് തുല്യമായ) ഔട്ട്പുട്ട് നൽകാൻ സഹായിക്കും. കൂടാതെ, ഓരോ ഘട്ടത്തിലും ഇൻപുട്ട് സ്വീകരിക്കാത്ത RNN യൂണിറ്റുകൾ ഉപയോഗിച്ച്, ചില പ്രാരംഭ സ്റ്റേറ്റ് വെക്ടർ എടുത്ത്, തുടർന്ന് ഔട്ട്പുട്ടുകളുടെ ഒരു സീക്വൻസ് ഉൽപ്പാദിപ്പിക്കാം.

ഈ നോട്ട്‌ബുക്കിൽ, നാം ടെക്സ്റ്റ് ജനറേറ്റ് ചെയ്യാൻ സഹായിക്കുന്ന ലളിതമായ ജനറേറ്റീവ് മോഡലുകളെ കേന്ദ്രീകരിക്കും. ലളിതത്വത്തിനായി, നാം **ക്യാരക്ടർ-ലെവൽ നെറ്റ്വർക്ക്** നിർമ്മിക്കാം, ഇത് അക്ഷരമുതൽ അക്ഷരം ടെക്സ്റ്റ് ജനറേറ്റ് ചെയ്യും. പരിശീലനത്തിനിടെ, നാം ഒരു ടെക്സ്റ്റ് കോർപ്പസ് എടുത്ത്, അത് അക്ഷര സീക്വൻസുകളായി വിഭജിക്കണം.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## അക്ഷര വാക്ക് നിർമ്മാണം

അക്ഷരനിര അടിസ്ഥാനത്തിലുള്ള ജനറേറ്റീവ് നെറ്റ്‌വർക്ക് നിർമ്മിക്കാൻ, വാക്കുകൾക്ക് പകരം ടെക്സ്റ്റ് വ്യക്തിഗത അക്ഷരങ്ങളായി വിഭജിക്കേണ്ടതുണ്ട്. മുമ്പ് ഉപയോഗിച്ച `TextVectorization` ലെയർ അത് ചെയ്യാൻ കഴിയില്ല, അതിനാൽ ഞങ്ങൾക്ക് രണ്ട് ഓപ്ഷനുകൾ ഉണ്ട്:

* കൈയോടെ ടെക്സ്റ്റ് ലോഡ് ചെയ്ത് ടോക്കനൈസേഷൻ ചെയ്യുക, [ഈ ഔദ്യോഗിക Keras ഉദാഹരണത്തിൽ](https://keras.io/examples/generative/lstm_character_level_text_generation/) കാണുന്നതുപോലെ
* അക്ഷരനിര ടോക്കനൈസേഷനായി `Tokenizer` ക്ലാസ് ഉപയോഗിക്കുക.

നാം രണ്ടാം ഓപ്ഷൻ തിരഞ്ഞെടുക്കും. `Tokenizer` വാക്കുകളായി ടോക്കനൈസ് ചെയ്യാനും ഉപയോഗിക്കാം, അതിനാൽ അക്ഷരനിര ടോക്കനൈസേഷനിൽ നിന്ന് വാക്ക് നിര ടോക്കനൈസേഷനിലേക്ക് എളുപ്പത്തിൽ മാറാൻ കഴിയും.

അക്ഷരനിര ടോക്കനൈസേഷൻ ചെയ്യാൻ, `char_level=True` പാരാമീറ്റർ നൽകണം:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

നാം ഒരു പ്രത്യേക ടോക്കൺ ഉപയോഗിച്ച് **ക്രമത്തിന്റെ അവസാനത്തെ** സൂചിപ്പിക്കാൻ ആഗ്രഹിക്കുന്നു, അതിനെ `<eos>` എന്ന് വിളിക്കും. അത് നമുക്ക് വാക്കുകളുടെ സമാഹാരത്തിൽ കൈമാറി ചേർക്കാം:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

ഇപ്പോൾ, ടെക്സ്റ്റ് നമ്പറുകളുടെ ക്രമങ്ങളായി എൻകോഡ് ചെയ്യാൻ, നാം ഉപയോഗിക്കാം:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## തലക്കെട്ടുകൾ സൃഷ്ടിക്കാൻ ജനറേറ്റീവ് RNN പരിശീലിപ്പിക്കൽ

നാം വാർത്താ തലക്കെട്ടുകൾ സൃഷ്ടിക്കാൻ RNN എങ്ങനെ പരിശീലിപ്പിക്കുമെന്ന് പറയുന്നത് ഇങ്ങനെ ആണ്. ഓരോ ഘട്ടത്തിലും, ഒരു തലക്കെട്ട് എടുത്ത് അത് RNN-ലേക്ക് നൽകും, ഓരോ ഇൻപുട്ട് അക്ഷരത്തിനും നെറ്റ്‌വർക്കിന് അടുത്ത ഔട്ട്പുട്ട് അക്ഷരം സൃഷ്ടിക്കാൻ ആവശ്യപ്പെടും:

!['HELLO' എന്ന വാക്കിന്റെ ഉദാഹരണ RNN സൃഷ്ടി കാണിക്കുന്ന ചിത്രം.](../../../../../translated_images/ml/rnn-generate.56c54afb52f9781d.webp)

നമ്മുടെ സീക്വൻസിലെ അവസാന അക്ഷരത്തിന്, നെറ്റ്‌വർക്കിന് `<eos>` ടോക്കൺ സൃഷ്ടിക്കാൻ ആവശ്യപ്പെടും.

നാം ഇവിടെ ഉപയോഗിക്കുന്ന ജനറേറ്റീവ് RNN-ന്റെ പ്രധാന വ്യത്യാസം, RNN-ന്റെ ഓരോ ഘട്ടത്തിലും ഔട്ട്പുട്ട് എടുക്കുന്നതാണ്, അവസാന സെല്ലിൽ നിന്നല്ല. ഇത് RNN സെല്ലിന് `return_sequences` പാരാമീറ്റർ നൽകിക്കൊണ്ട് സാധ്യമാക്കാം.

അതിനാൽ, പരിശീലന സമയത്ത്, നെറ്റ്‌വർക്കിന്റെ ഇൻപുട്ട് ഒരു നിശ്ചിത നീളമുള്ള എൻകോഡുചെയ്ത അക്ഷരങ്ങളുടെ സീക്വൻസായിരിക്കും, ഔട്ട്പുട്ട് അതേ നീളമുള്ള, പക്ഷേ ഒരു ഘടകം മാറ്റിയതും `<eos>`-ൽ അവസാനിക്കുന്നതുമായ സീക്വൻസായിരിക്കും. മിനിബാച്ച് പല സീക്വൻസുകളും ഉൾക്കൊള്ളും, അതിനാൽ എല്ലാ സീക്വൻസുകളും സജ്ജമാക്കാൻ **padding** ഉപയോഗിക്കേണ്ടതുണ്ട്.

ഡാറ്റാസെറ്റ് മാറ്റാൻ സഹായിക്കുന്ന ഫംഗ്ഷനുകൾ സൃഷ്ടിക്കാം. മിനിബാച്ച് തലത്തിൽ സീക്വൻസുകൾ പാഡ് ചെയ്യേണ്ടതിനാൽ, ആദ്യം `.batch()` വിളിച്ച് ഡാറ്റാസെറ്റ് ബാച്ച് ചെയ്യുകയും, പിന്നീട് `map` ഉപയോഗിച്ച് മാറ്റം വരുത്തുകയും ചെയ്യും. അതിനാൽ, മാറ്റം വരുത്തുന്ന ഫംഗ്ഷൻ മുഴുവൻ മിനിബാച്ച് ഒരു പാരാമീറ്ററായി സ്വീകരിക്കും:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

നാം ഇവിടെ ചെയ്യുന്ന ചില പ്രധാന കാര്യങ്ങൾ:
* ആദ്യം സ്ട്രിംഗ് ടെൻസറിൽ നിന്ന് യഥാർത്ഥ ടെക്സ്റ്റ് എടുക്കുന്നു
* `text_to_sequences` സ്ട്രിംഗുകളുടെ ലിസ്റ്റ് ഇന്റിജർ ടെൻസറുകളുടെ ലിസ്റ്റായി മാറ്റുന്നു
* `pad_sequences` ആ ടെൻസറുകളെ അവരുടെ പരമാവധി നീളത്തിലേക്ക് പാഡ് ചെയ്യുന്നു
* അവസാനം എല്ലാ അക്ഷരങ്ങളും വൺ-ഹോട്ട് എൻകോഡ് ചെയ്യുകയും, ഷിഫ്റ്റിംഗ് ചെയ്യുകയും `<eos>` ചേർക്കുകയും ചെയ്യുന്നു. വൺ-ഹോട്ട് എൻകോഡ് ചെയ്ത അക്ഷരങ്ങൾ എന്തിന് വേണ്ടിയാണെന്ന് നമുക്ക് ഉടൻ കാണാം

എങ്കിലും, ഈ ഫംഗ്ഷൻ **Pythonic** ആണ്, അതായത് ഇത് സ്വയം Tensorflow കംപ്യൂട്ടേഷണൽ ഗ്രാഫിലേക്ക് മാറ്റാൻ കഴിയില്ല. ഈ ഫംഗ്ഷൻ നേരിട്ട് `Dataset.map` ഫംഗ്ഷനിൽ ഉപയോഗിക്കാൻ ശ്രമിച്ചാൽ പിശകുകൾ ഉണ്ടാകും. അതിനാൽ ഈ Pythonic കോൾ `py_function` റാപ്പർ ഉപയോഗിച്ച് ഉൾപ്പെടുത്തേണ്ടതുണ്ട്:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **കുറിപ്പ്**: Pythonic-ഉം Tensorflow ട്രാൻസ്ഫർമേഷൻ ഫങ്ഷനുകളും തമ്മിലുള്ള വ്യത്യാസം കുറച്ച് സങ്കീർണ്ണമായി തോന്നാം, കൂടാതെ dataset-നെ `fit`-ന് മുമ്പ് സാധാരണ Python ഫങ്ഷനുകൾ ഉപയോഗിച്ച് ട്രാൻസ്ഫോം ചെയ്യാത്തതെന്തുകൊണ്ടെന്ന് നിങ്ങൾ ചോദിച്ചേക്കാം. ഇത് തീർച്ചയായും ചെയ്യാവുന്നതാണ്, എന്നാൽ `Dataset.map` ഉപയോഗിക്കുന്നത് വലിയ ഗുണം നൽകുന്നു, കാരണം ഡാറ്റ ട്രാൻസ്ഫർമേഷൻ പൈപ്പ്‌ലൈൻ Tensorflow കംപ്യൂട്ടേഷണൽ ഗ്രാഫ് ഉപയോഗിച്ച് പ്രവർത്തിക്കുന്നു, ഇത് GPU കംപ്യൂട്ടേഷനുകളുടെ പ്രയോജനം എടുക്കുകയും CPU/GPU-വിനിടയിൽ ഡാറ്റ കൈമാറ്റം കുറയ്ക്കുകയും ചെയ്യുന്നു.

ഇപ്പോൾ നാം നമ്മുടെ ജനറേറ്റർ നെറ്റ്‌വർക്ക് നിർമ്മിച്ച് പരിശീലനം ആരംഭിക്കാം. ഇത് മുമ്പത്തെ യൂണിറ്റിൽ ചർച്ച ചെയ്ത ഏതെങ്കിലും റിക്കറന്റ് സെല്ലിൽ അടിസ്ഥാനമാക്കാവുന്നതാണ് (സിംപിൾ, LSTM അല്ലെങ്കിൽ GRU). നമ്മുടെ ഉദാഹരണത്തിൽ LSTM ഉപയോഗിക്കും.

നെറ്റ്‌വർക്ക് കാറക്ടറുകൾ ഇൻപുട്ടായി സ്വീകരിക്കുന്നതിനാൽ, വാക്കുകളുടെ വലുപ്പം വളരെ ചെറുതാണ്, അതിനാൽ embedding ലെയർ ആവശ്യമില്ല, one-hot-encoded ഇൻപുട്ട് നേരിട്ട് LSTM സെല്ലിലേക്ക് പോകാം. ഔട്ട്പുട്ട് ലെയർ LSTM ഔട്ട്പുട്ട് one-hot-encoded ടോക്കൺ നമ്പറുകളായി മാറ്റുന്ന `Dense` ക്ലാസിഫയർ ആയിരിക്കും.

കൂടാതെ, വ്യത്യസ്ത നീളമുള്ള സീക്വൻസുകളുമായി കൈകാര്യം ചെയ്യുന്നതിനാൽ, പാഡുചെയ്ത ഭാഗം അവഗണിക്കുന്ന ഒരു മാസ്ക് സൃഷ്ടിക്കാൻ `Masking` ലെയർ ഉപയോഗിക്കാം. `<eos>` ടോക്കണിന് ശേഷം വരുന്ന എല്ലാം നമ്മൾ വളരെ ശ്രദ്ധിക്കാത്തതിനാൽ ഇത് കർശനമായി ആവശ്യമായില്ല, പക്ഷേ ഈ ലെയർ തരം ഉപയോഗിച്ച് അനുഭവം നേടുന്നതിനായി ഇത് ഉപയോഗിക്കും. `input_shape` `(None, vocab_size)` ആയിരിക്കും, ഇവിടെ `None` വ്യത്യസ്ത നീളമുള്ള സീക്വൻസിനെ സൂചിപ്പിക്കുന്നു, ഔട്ട്പുട്ട് ഷേപ്പ് `(None, vocab_size)` ആണെന്നും `summary`-ൽ കാണാം:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## ഔട്ട്പുട്ട് സൃഷ്ടിക്കൽ

ഇപ്പോൾ മോഡൽ പരിശീലിപ്പിച്ചതിനുശേഷം, അതുപയോഗിച്ച് ചില ഔട്ട്പുട്ടുകൾ സൃഷ്ടിക്കാൻ ആഗ്രഹിക്കുന്നു. ആദ്യം, ടോക്കൺ നമ്പറുകളുടെ ശ്രേണിയാൽ പ്രതിനിധീകരിച്ച ടെക്സ്റ്റ് ഡീകോഡ് ചെയ്യാനുള്ള ഒരു മാർഗം വേണം. ഇതിന്, `tokenizer.sequences_to_texts` ഫംഗ്ഷൻ ഉപയോഗിക്കാമായിരുന്നു; എന്നാൽ, ഇത് കറക്റ്റർ-ലെവൽ ടോക്കണൈസേഷനുമായി നല്ല രീതിയിൽ പ്രവർത്തിക്കുന്നില്ല. അതിനാൽ, ടോക്കണൈസറിൽ നിന്നുള്ള ടോക്കൺ ഡിക്ഷണറി (പേര് `word_index`) എടുത്ത്, അതിന്റെ റിവേഴ്സ് മാപ്പ് നിർമ്മിച്ച്, ഞങ്ങൾ സ്വന്തം ഡീകോഡിംഗ് ഫംഗ്ഷൻ എഴുതും:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

ഇപ്പോൾ, നമുക്ക് ജനറേഷൻ ചെയ്യാം. നാം ചില സ്ട്രിംഗ് `start` ഉപയോഗിച്ച് തുടങ്ങും, അതിനെ ഒരു സീക്വൻസ് `inp` ആയി എൻകോഡ് ചെയ്യും, പിന്നീട് ഓരോ ഘട്ടത്തിലും അടുത്ത അക്ഷരം പ്രവചിക്കാൻ നമ്മുടെ നെറ്റ്‌വർക്ക് വിളിക്കും.

നെറ്റ്‌വർക്കിന്റെ ഔട്ട്പുട്ട് `out` എന്നത് `vocab_size` ഘടകങ്ങളുള്ള ഒരു വെക്ടറാണ്, ഓരോ ടോക്കണിന്റെയും സാധ്യതകൾ പ്രതിനിധീകരിക്കുന്നു, ഏറ്റവും സാധ്യതയുള്ള ടോക്കൺ നമ്പർ കണ്ടെത്താൻ `argmax` ഉപയോഗിക്കാം. പിന്നീട് ആ അക്ഷരം ജനറേറ്റുചെയ്ത ടോക്കൺ ലിസ്റ്റിൽ ചേർക്കുകയും, ജനറേഷൻ തുടരുകയും ചെയ്യും. ഒരു അക്ഷരം ജനറേറ്റ് ചെയ്യാനുള്ള ഈ പ്രക്രിയ `size` തവണ ആവർത്തിച്ച് ആവശ്യമായ അക്ഷരങ്ങളുടെ എണ്ണം സൃഷ്ടിക്കും, കൂടാതെ `eos_token` കണ്ടുപിടിച്ചാൽ നേരത്തെ അവസാനിപ്പിക്കും.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## പരിശീലനത്തിനിടെ സാമ്പിളിംഗ് ഔട്ട്പുട്ട്

*ശുദ്ധത* പോലുള്ള ഉപകാരപ്രദമായ മെട്രിക്കുകൾ ഇല്ലാത്തതിനാൽ, നമ്മുടെ മോഡൽ മെച്ചപ്പെടുന്നുണ്ടെന്ന് കാണാനുള്ള ഏക മാർഗം പരിശീലനത്തിനിടെ സൃഷ്ടിച്ച സ്ട്രിംഗ് **സാമ്പിളിംഗ്** ചെയ്യലാണ്. ഇത് ചെയ്യാൻ, നാം **കോൾബാക്കുകൾ** ഉപയോഗിക്കും, അഥവാ `fit` ഫംഗ്ഷനിലേക്ക് പാസ്സ് ചെയ്യാവുന്ന, പരിശീലനത്തിനിടെ കാലക്രമേണ വിളിക്കപ്പെടുന്ന ഫംഗ്ഷനുകൾ.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


ഈ ഉദാഹരണം ഇതിനകം തന്നെ നല്ലൊരു ടെക്സ്റ്റ് സൃഷ്ടിക്കുന്നു, പക്ഷേ ഇത് പലവിധത്തിൽ മെച്ചപ്പെടുത്താൻ കഴിയും:
* **കൂടുതൽ ടെക്സ്റ്റ്**. ഞങ്ങൾ നമ്മുടെ ടാസ്കിനായി തലക്കെട്ടുകൾ മാത്രം ഉപയോഗിച്ചിട്ടുണ്ട്, പക്ഷേ നിങ്ങൾ പൂർണ്ണ ടെക്സ്റ്റുമായി പരീക്ഷണം നടത്താൻ ആഗ്രഹിക്കാം. RNN-കൾ ദീർഘമായ സീക്വൻസുകൾ കൈകാര്യം ചെയ്യുന്നതിൽ അത്ര നല്ലതല്ലെന്ന് ഓർക്കുക, അതിനാൽ അവയെ ചെറുതായുള്ള വാക്യങ്ങളായി വിഭജിക്കുക അല്ലെങ്കിൽ നിശ്ചിത നീളമുള്ള സീക്വൻസിൽ (ഉദാ., `num_chars` എന്ന മുൻകൂട്ടി നിശ്ചയിച്ച മൂല്യം, 256 പോലുള്ളത്) പരിശീലനം നടത്തുന്നത് ബുദ്ധിമുട്ടില്ല. മുകളിൽ നൽകിയ ഉദാഹരണം [അധികൃത Keras ട്യൂട്ടോറിയൽ](https://keras.io/examples/generative/lstm_character_level_text_generation/) പ്രചോദനമായി ഉപയോഗിച്ച് ഇത്തരത്തിലുള്ള ആർക്കിടെക്ചറിലേക്ക് മാറ്റാൻ ശ്രമിക്കാം.
* **മൾട്ടിലെയർ LSTM**. 2 അല്ലെങ്കിൽ 3 ലെയറുകളുടെ LSTM സെല്ലുകൾ പരീക്ഷിക്കുന്നത് ഉചിതമാണ്. മുൻ യൂണിറ്റിൽ പറഞ്ഞതുപോലെ, ഓരോ LSTM ലെയറും ടെക്സ്റ്റിൽ നിന്നുള്ള പ്രത്യേക പാറ്റേണുകൾ എടുക്കുന്നു, കറക്റ്റർ-ലെവൽ ജനറേറ്ററിന്റെ കാര്യത്തിൽ താഴത്തെ LSTM ലെയർ സില്ലബിളുകൾ എടുക്കുന്നതിന് ഉത്തരവാദിയാകും, മുകളിലെ ലെയറുകൾ വാക്കുകളും വാക്കുകളുടെ സംയോജനങ്ങളും എടുക്കുന്നതിന്. ഇത് LSTM കൺസ്ട്രക്ടറിലേക്ക് ലെയറുകളുടെ എണ്ണം പാരാമീറ്ററായി നൽകുന്നതിലൂടെ എളുപ്പത്തിൽ നടപ്പിലാക്കാം.
* **GRU യൂണിറ്റുകൾ** ഉപയോഗിച്ച് പരീക്ഷണം നടത്താനും ഏത് മെച്ചമാണ് എന്ന് കാണാനും, കൂടാതെ **വിവിധ ഹിഡൻ ലെയർ വലുപ്പങ്ങൾ** പരീക്ഷിക്കാനും നിങ്ങൾ ആഗ്രഹിക്കാം. വളരെ വലിയ ഹിഡൻ ലെയർ ഓവർഫിറ്റിങ്ങിന് കാരണമാകാം (ഉദാ., നെറ്റ്‌വർക്ക് കൃത്യമായ ടെക്സ്റ്റ് പഠിക്കും), ചെറുതായുള്ള വലുപ്പം നല്ല ഫലം നൽകാതിരിക്കാം.


## സോഫ്റ്റ് ടെക്സ്റ്റ് ജനറേഷൻ ಮತ್ತು ടെംപറേച്ചർ

മുൻപ് `generate` എന്ന നിർവചനത്തിൽ, നാം എപ്പോഴും ഏറ്റവും ഉയർന്ന സാധ്യതയുള്ള അക്ഷരത്തെ അടുത്ത അക്ഷരമായി എടുത്തിരുന്നു. ഇതിന്റെ ഫലമായി, ടെക്സ്റ്റ് പലപ്പോഴും ഒരേ അക്ഷരക്രമങ്ങൾ വീണ്ടും വീണ്ടും "സൈക്ല്" ചെയ്യുന്നതായി കണ്ടു, ഉദാഹരണത്തിന് താഴെ കാണുന്ന പോലെ:
```
today of the second the company and a second the company ...
```

എങ്കിലും, അടുത്ത അക്ഷരത്തിനുള്ള സാധ്യത വിതരണത്തെ നോക്കിയാൽ, ഏറ്റവും ഉയർന്ന ചില സാധ്യതകളുടെ വ്യത്യാസം വലിയതല്ലായിരിക്കാം, ഉദാഹരണത്തിന് ഒരു അക്ഷരത്തിന് 0.2 സാധ്യതയുണ്ടെങ്കിൽ, മറ്റൊന്ന് 0.19 ആയിരിക്കാം. ഉദാഹരണത്തിന് '*play*' എന്ന ക്രമത്തിൽ അടുത്ത അക്ഷരം സ്പേസ് ആകാമോ, അല്ലെങ്കിൽ **e** (പദമായ *player* ൽപോലെ) ആകാമോ എന്നത് സമാനമായ സാധ്യതയുള്ളതാണ്.

ഇത് നമ്മെ ഈ നിഗമനത്തിലേക്ക് നയിക്കുന്നു: ഏറ്റവും ഉയർന്ന സാധ്യതയുള്ള അക്ഷരം തിരഞ്ഞെടുക്കുന്നത് എപ്പോഴും "ന്യായമായ" കാര്യമല്ല, കാരണം രണ്ടാം ഉയർന്ന സാധ്യതയുള്ളത് തിരഞ്ഞെടുക്കുന്നതും അർത്ഥവത്തായ ടെക്സ്റ്റിലേക്ക് നയിക്കാം. നെറ്റ്‌വർക്ക് ഔട്ട്പുട്ട് നൽകുന്ന സാധ്യത വിതരണത്തിൽ നിന്നാണ് അക്ഷരങ്ങൾ **സാമ്പിൾ** ചെയ്യുന്നത് കൂടുതൽ ബുദ്ധിമുട്ടുള്ളതും ഉചിതവുമാണ്.

ഈ സാമ്പിളിംഗ് `np.multinomial` ഫംഗ്ഷൻ ഉപയോഗിച്ച് ചെയ്യാം, ഇത് **മൾട്ടിനോമിയൽ വിതരണ**ം നടപ്പിലാക്കുന്നു. താഴെ ഈ **സോഫ്റ്റ്** ടെക്സ്റ്റ് ജനറേഷൻ നടപ്പിലാക്കുന്ന ഫംഗ്ഷൻ നിർവചിച്ചിരിക്കുന്നു:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

നാം **temperature** എന്ന ഒരു പുതിയ പാരാമീറ്റർ പരിചയപ്പെടുത്തി, ഇത് ഏറ്റവും ഉയർന്ന സാധ്യതയോട് എത്രമാത്രം കർശനമായി പാലിക്കണമെന്ന് സൂചിപ്പിക്കാൻ ഉപയോഗിക്കുന്നു. temperature 1.0 ആണെങ്കിൽ, നാം ശരാശരി മൾട്ടിനോമിയൽ സാമ്പ്ലിംഗ് നടത്തുന്നു, temperature അനന്തതയിലേക്ക് പോകുമ്പോൾ - എല്ലാ സാധ്യതകളും സമാനമാകുന്നു, പിന്നെ നാം യാദൃച്ഛികമായി അടുത്ത അക്ഷരം തിരഞ്ഞെടുക്കുന്നു. താഴെ കൊടുത്ത ഉദാഹരണത്തിൽ temperature വളരെ കൂടുമ്പോൾ വാചകം അർത്ഥരഹിതമാകുന്നത് കാണാം, കൂടാതെ അത് 0-ന് അടുത്ത് പോകുമ്പോൾ "cycled" കർശനമായി സൃഷ്ടിച്ച വാചകത്തെപ്പോലെ തോന്നുന്നു.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കപ്പെടണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗത്തിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
